# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

<Client: 'tcp://127.0.0.1:43981' processes=1 threads=4, memory=1.86 GiB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [6]:
df.head()

id      name         x         y
0   988   Norbert -0.742721 -0.277954
1  1025       Bob  0.603313 -0.161292
2   992     Alice -0.049408  0.573142
3  1029       Bob -0.122566  0.533852
4  1032  Patricia  0.476066 -0.006417

Let's look at some statistics on the data

In [7]:
df.describe().compute()

id             x             y
count  2.592000e+06  2.592000e+06  2.592000e+06
mean   9.999909e+02 -1.752288e-04  1.272128e-04
std    3.163993e+01  5.772766e-01  5.773655e-01
min    8.370000e+02 -9.999999e-01 -9.999995e-01
25%    9.790000e+02 -4.924166e-01 -4.938494e-01
50%    1.000000e+03  9.977439e-03  4.362202e-03
75%    1.022000e+03  5.070134e-01  5.083363e-01
max    1.160000e+03  9.999979e-01  9.999995e-01

# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
1032,Edith,0.341158963292153
1025,Yvonne,-0.0596561961788608
996,Hannah,-0.4598038238105364
1015,Norbert,-0.6893967021653444
976,Hannah,0.4339578272105588
1002,Dan,0.3519233500902228
917,Xavier,-0.928241343897473
1036,Hannah,-0.5115504865546654
972,Oliver,-0.3808144336718926


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

id      name         x         y
0   988   Norbert -0.742721 -0.277954
1  1025       Bob  0.603313 -0.161292
2   992     Alice -0.049408  0.573142
3  1029       Bob -0.122566  0.533852
4  1032  Patricia  0.476066 -0.006417

Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

id      name         x         y
0   988   Norbert -0.742721 -0.277954
1  1025       Bob  0.603313 -0.161292
2   992     Alice -0.049408  0.573142
3  1029       Bob -0.122566  0.533852
4  1032  Patricia  0.476066 -0.006417

In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

Empty DataFrame
Columns: [id, name, x, y]
Index: []

In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

id     name         x   y
0  1032    Edith  0.341159 NaN
1  1025   Yvonne -0.059656 NaN
2   996   Hannah -0.459804 NaN
3  1015  Norbert -0.689397 NaN
4   976   Hannah  0.433958 NaN

# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-604fd047-a660-4c67-87ad-60569554e79e')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

id      name         x         y
0       988   Norbert -0.742721 -0.277954
1      1025       Bob  0.603313 -0.161292
2       992     Alice -0.049408  0.573142
3      1029       Bob -0.122566  0.533852
4      1032  Patricia  0.476066 -0.006417
...     ...       ...       ...       ...
86395   927     Alice  0.051035  0.051330
86396   968    George -0.389181  0.096867
86397  1039     Alice  0.396751  0.688604
86398   996  Patricia -0.042164 -0.924152
86399   956       Tim  0.854212  0.858070

[86400 rows x 4 columns]

Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-25.csv',
 'data/2000-01-20.csv',
 'data/2000-01-29.csv',
 'data/2000-01-02.csv',
 'data/2000-01-19.csv',
 'data/2000-01-23.csv',
 'data/2000-01-10.csv',
 'data/2000-01-21.csv',
 'data/2000-01-17.csv',
 'data/2000-01-04.csv',
 'data/2000-01-27.csv',
 'data/2000-01-22.csv',
 'data/2000-01-14.csv',
 'data/2000-01-11.csv',
 'data/2000-01-13.csv',
 'data/2000-01-08.csv',
 'data/2000-01-09.csv',
 'data/2000-01-06.csv',
 'data/2000-01-01.csv',
 'data/2000-01-07.csv',
 'data/2000-01-12.csv',
 'data/2000-01-16.csv',
 'data/2000-01-26.csv',
 'data/2000-01-24.csv',
 'data/2000-01-18.csv',
 'data/2000-01-15.csv',
 'data/2000-01-03.csv',
 'data/2000-01-30.csv',
 'data/2000-01-28.csv',
 'data/2000-01-05.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-6958e08a-f47f-4da2-9ceb-cb995eab99bf'),
 Delayed('read_data-30f33e21-323e-49c0-ae36-a7c6289d8ada'),
 Delayed('read_data-2fdfe85e-79e1-417d-af2d-3a577fe15975'),
 Delayed('read_data-72b35641-90b5-4518-bbac-fa9c9024c756'),
 Delayed('read_data-e3adc855-9df0-4985-87fd-f95e3f2d10b7'),
 Delayed('read_data-f070f86c-bff6-448e-abe0-50baaf9282b0'),
 Delayed('read_data-f4ed9f6d-c5ae-44aa-ba0e-a2eaf2cd749a'),
 Delayed('read_data-2a55c497-9a5a-4474-8dca-fc243ee5a5bf'),
 Delayed('read_data-1fb346a5-4a27-4772-ab6b-94419f328ae0'),
 Delayed('read_data-72610d5f-2847-4afb-9c86-af08217797d2'),
 Delayed('read_data-f3bbcc5b-c2f6-4a5f-8d3e-1d50fb30dc69'),
 Delayed('read_data-52b113b8-1692-4ff9-86b4-cb65e066e1c3'),
 Delayed('read_data-ff401421-8ccf-4e29-bf70-8b63ed4e8b90'),
 Delayed('read_data-ebe81647-e84f-4377-ba1c-26f220aed7e3'),
 Delayed('read_data-dabf5c6c-e459-4f89-9a02-4b4a11879708'),
 Delayed('read_data-c7b3408b-2cec-41e1-9553-fb9a24a338b0'),
 Delayed('read_data-fbd802e1-f886-4035-a

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
                int64  object  float64  float64
                  ...     ...      ...      ...
...               ...     ...      ...      ...
                  ...     ...      ...      ...
                  ...     ...      ...      ...
Dask Name: from-delayed, 60 tasks

Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

id    name         x         y
0   976  Oliver  0.628767  0.765093
1  1053   Sarah -0.047006 -0.955109
2  1049   Quinn -0.032074 -0.099608
3  1005   Frank -0.255920  0.963524
4   993  Ursula  0.980263 -0.875488

In [26]:
df.describe().compute()

id             x             y
count  2.505600e+06  2.505600e+06  2.419200e+06
mean   9.999870e+02 -1.615087e-04  7.238821e-05
std    3.163868e+01  5.772576e-01  5.774155e-01
min    8.370000e+02 -9.999999e-01 -9.999995e-01
25%    9.790000e+02 -4.924166e-01 -4.938494e-01
50%    1.000000e+03  9.977439e-03  4.362202e-03
75%    1.022000e+03  5.070134e-01  5.083363e-01
max    1.160000e+03  9.999979e-01  9.999995e-01

## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.